In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import scipy.fftpack
from scipy.linalg import toeplitz
from scipy.signal import fftconvolve
import itertools
import warnings
import os
import glob
import torch
import torch.nn as nn
import tqdm
import librosa
import time
import random
import soundfile

In [ ]:
## FUNCTIONS FOR EVALUATION OF MODEL, TO CALCULATE SDR, SIR AND SAR, TAKEN  FROM THE BSS EVAL TOOLBOX, AS MENTIONED IN OUR PAPER
def bss_eval_sources(reference_sources, estimated_sources):
    if estimated_sources.ndim == 1:
        estimated_sources = estimated_sources[np.newaxis, :]
    if reference_sources.ndim == 1:
        reference_sources = reference_sources[np.newaxis, :]
    if reference_sources.size == 0 or estimated_sources.size == 0:
        return np.array([]), np.array([]), np.array([]), np.array([])
    nsrc = estimated_sources.shape[0]
    sdr = np.empty((nsrc, nsrc))
    sir = np.empty((nsrc, nsrc))
    sar = np.empty((nsrc, nsrc))
    for jest in range(nsrc):
        for jtrue in range(nsrc):
            s_true, e_spat, e_interf, e_artif = \
                    _bss_decomp_mtifilt(reference_sources,
                                        estimated_sources[jest],
                                        jtrue, 512)
            sdr[jest, jtrue], sir[jest, jtrue], sar[jest, jtrue] = \
                    _bss_source_crit(s_true, e_spat, e_interf, e_artif)
    perms = list(itertools.permutations(list(range(nsrc))))
    mean_sir = np.empty(len(perms))
    dum = np.arange(nsrc)
    for (i, perm) in enumerate(perms):
        mean_sir[i] = np.mean(sir[perm, dum])
    popt = perms[np.argmax(mean_sir)]
    idx = (popt, dum)
    return (sdr[idx], sir[idx], sar[idx], np.asarray(popt))

def _bss_decomp_mtifilt(reference_sources, estimated_source, j, flen):
    nsampl = estimated_source.size
    s_true = np.hstack((reference_sources[j], np.zeros(flen - 1)))
    e_spat = _project(reference_sources[j, np.newaxis, :], estimated_source,
                      flen) - s_true
    e_interf = _project(reference_sources,
                        estimated_source, flen) - s_true - e_spat
    e_artif = -s_true - e_spat - e_interf
    e_artif[:nsampl] += estimated_source
    return (s_true, e_spat, e_interf, e_artif)


def _bss_source_crit(s_true, e_spat, e_interf, e_artif):
    s_filt = s_true + e_spat
    sdr = _safe_db(np.sum(s_filt**2), np.sum((e_interf + e_artif)**2))
    sir = _safe_db(np.sum(s_filt**2), np.sum(e_interf**2))
    sar = _safe_db(np.sum((s_filt + e_interf)**2), np.sum(e_artif**2))
    return (sdr, sir, sar)


def _safe_db(num, den):
    if den == 0:
        return np.Inf
    return 10 * np.log10(num / den)


def _project(reference_sources, estimated_source, flen):
    nsrc = reference_sources.shape[0]
    nsampl = reference_sources.shape[1]
    reference_sources = np.hstack((reference_sources,
                                   np.zeros((nsrc, flen - 1))))
    estimated_source = np.hstack((estimated_source, np.zeros(flen - 1)))
    n_fft = int(2**np.ceil(np.log2(nsampl + flen - 1.)))
    sf = scipy.fftpack.fft(reference_sources, n=n_fft, axis=1)
    sef = scipy.fftpack.fft(estimated_source, n=n_fft)
    G = np.zeros((nsrc * flen, nsrc * flen))
    for i in range(nsrc):
        for j in range(nsrc):
            ssf = sf[i] * np.conj(sf[j])
            ssf = np.real(scipy.fftpack.ifft(ssf))
            ss = toeplitz(np.hstack((ssf[0], ssf[-1:-flen:-1])),
                          r=ssf[:flen])
            G[i * flen: (i+1) * flen, j * flen: (j+1) * flen] = ss
            G[j * flen: (j+1) * flen, i * flen: (i+1) * flen] = ss.T
    D = np.zeros(nsrc * flen)
    for i in range(nsrc):
        ssef = sf[i] * np.conj(sef)
        ssef = np.real(scipy.fftpack.ifft(ssef))
        D[i * flen: (i+1) * flen] = np.hstack((ssef[0], ssef[-1:-flen:-1]))
    try:
        C = np.linalg.solve(G, D).reshape(flen, nsrc, order='F')
    except np.linalg.linalg.LinAlgError:
        C = np.linalg.lstsq(G, D)[0].reshape(flen, nsrc, order='F')
    sproj = np.zeros(nsampl + flen - 1)
    for i in range(nsrc):
        sproj += fftconvolve(C[:, i], reference_sources[i])[:nsampl + flen - 1]
    return sproj


def bss_eval(mixed_wav, src1_wav, src2_wav, pred_src1_wav, pred_src2_wav):
    len = pred_src1_wav.shape[0]
    src1_wav = src1_wav[:len]
    src2_wav = src2_wav[:len]
    mixed_wav = mixed_wav[:len]
    bss_eval_sources
    sdr, sir, sar, _ = bss_eval_sources(np.array([src1_wav, src2_wav]),
                                 np.array([pred_src1_wav, pred_src2_wav]))
    sdr_mixed, _, _, _ = bss_eval_sources(np.array([src1_wav, src2_wav]),
                                   np.array([mixed_wav, mixed_wav]))
    nsdr = sdr - sdr_mixed
    return nsdr, sir, sar, len

In [ ]:
## RANDOMISING TRAINING DATA AND ENSURING THAT BOTH MALE AND FEMALE AUDIO SAMPLES GO INTO TRAINING

male = ["abjones","bobon","bug","davidson","fdps","geniusturtle","jmzen","Kenshin","khair","leon","stool"] #LIST OF ALL MALE VOICES
male=np.array(male)
female =["amy", "Ani", "annar","ariel","heycat","tammy","titon","yifen"] #LIST OF ALL FEMALE VOICES
female=np.array(female)
Nsample=len(male)
Ntest=0.2 * Nsample
Ntrain=int(0.8*Nsample)
index=np.arange(Nsample)
index_random=np.random.permutation(index)
train_index=index_random[:Ntrain]
test_index=index_random[Ntrain:]
male_train=[]
male_test=[]
for i in range(Ntrain):
  male_train.append(male[train_index[i]])
  
for i in range(Nsample-Ntrain):
  male_test.append(male[test_index[i]])

Nsample=len(female)
Ntest=0.2 * Nsample
Ntrain=int(0.8*Nsample)
index=np.arange(Nsample)
index_random=np.random.permutation(index)
train_index=index_random[:Ntrain]
test_index=index_random[Ntrain:]
female_train=[]
female_test=[]
for i in range(Ntrain):
  female_train.append(female[train_index[i]])
for i in range(Nsample-Ntrain):
  female_test.append(female[test_index[i]])

print(male_train)
print(male_test)
print(female_train)
print(female_test)
test_data = male_test+female_test
random.shuffle(test_data)
print(test_data)
train_d = male_train+female_train
random.shuffle(train_d)
print(train_d)

['Kenshin', 'bobon', 'davidson', 'jmzen', 'fdps', 'abjones', 'leon', 'bug']
['khair', 'geniusturtle', 'stool']
['Ani', 'tammy', 'titon', 'ariel', 'heycat', 'annar']
['amy', 'yifen']
['stool', 'amy', 'khair', 'yifen', 'geniusturtle']
['bug', 'leon', 'Kenshin', 'davidson', 'fdps', 'heycat', 'Ani', 'abjones', 'jmzen', 'annar', 'ariel', 'titon', 'tammy', 'bobon']


In [ ]:
#functions to generate data for training and testing, calculate short time fourier transform and inverse stft

def data_generator(train_d,train):
    for wav in glob.glob('/content/drive/MyDrive/wavfile/*'):
        f = os.path.split(wav)[1]
        if (f.startswith(train_d[0]) or f.startswith(train_d[1]) or f.startswith(train_d[2]) or f.startswith(train_d[2]) or f.startswith(train_d[3]) or f.startswith(train_d[4]) or f.startswith(train_d[5]) or f.startswith(train_d[6]) or f.startswith(train_d[7]) or f.startswith(train_d[8]) or f.startswith(train_d[9]) or f.startswith(train_d[10]) or f.startswith(train_d[11]) or f.startswith(train_d[12]) or f.startswith(train_d[13])) == train:
            origin_source, sampling_rate = librosa.load(wav, sr=None, mono=False)  
            
            source_resampled = librosa.resample(origin_source, sampling_rate, 8000)
            mixed_source_origin = librosa.to_mono(source_resampled)
            resampled_left = source_resampled[0]
            resampled_right = source_resampled[1]
            mixed_magnitude_spectrum = np.abs(spectrum(mixed_source_origin))
            right_magnitude_spectrum = np.abs(spectrum(np.asfortranarray(resampled_right)))
            left_magnitude_spectrum = np.abs(spectrum(np.asfortranarray(resampled_left))) 
            max_value = np.max(mixed_magnitude_spectrum)
            mixed_phase = np.angle(spectrum(mixed_source_origin)) # won't be used in training, preserving this for obtaining the predicted wavfile from the predicted feature masks
            #Normalizing spectrograms
            mixed_magnitude_spectrum = mixed_magnitude_spectrum / max_value
            right_magnitude_spectrum = right_magnitude_spectrum / max_value
            left_magnitude_spectrum = left_magnitude_spectrum / max_value
            yield origin_source[0, :], origin_source[1, :], librosa.to_mono(origin_source), left_magnitude_spectrum, right_magnitude_spectrum, mixed_magnitude_spectrum, max_value, mixed_phase

def spectrum(wav):
    return librosa.stft(wav, n_fft = 1024, hop_length = 256)

def to_wav(mag, phase):
    mat = mag * np.exp(1.j * phase)
    return np.array(librosa.istft(mat, hop_length = 256))

In [ ]:
# defining our model 

class Conv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1):
        super(Conv, self).__init__()
        self.model = nn.Sequential(
                nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride,
                          padding=(kernel_size - 1) // 2),
                nn.BatchNorm2d(num_features=out_channels),
                nn.ReLU())
    def forward(self, x):
        return self.model(x) 

class Hourglass(nn.Module): #one hourglass module
    def __init__(self, depth, output_channels, add_channels):
        super(Hourglass, self).__init__()
        next_channels = output_channels + add_channels
        self.model = nn.Sequential(nn.MaxPool2d(kernel_size=2),
                                   Conv(output_channels, next_channels),
                                   Hourglass(depth - 1, next_channels,
                                             add_channels) if depth else Conv(
                                       next_channels, next_channels),
                                   Conv(next_channels, output_channels),
                                   nn.UpsamplingNearest2d(scale_factor=2)) 
        self.skip = Conv(output_channels, output_channels)
    def forward(self, x):
        return self.skip(x) + self.model(x)

class StackedHourglassNet(nn.Module): #stacking the hourglasses to build the final structure
    def __init__(self, num_stacks, channels, output_channels, add_channels):
        super(StackedHourglassNet, self).__init__()
        self.num_stacks = num_stacks
        self.prepare = nn.Sequential(Conv(1, 64, kernel_size=7, stride=1),
                                     Conv(64, 128), 
                                     Conv(128, 128),
                                     Conv(128, channels))
        self.hourglass = nn.ModuleList(
            nn.Sequential(
                Hourglass(depth=4, output_channels=channels, add_channels=add_channels),
                Conv(channels, channels),
                Conv(channels, channels, 1)) for i in range(num_stacks))
        self.output = nn.ModuleList(Conv(channels, output_channels) for i in range(num_stacks))
        self.next = nn.ModuleList(Conv(channels, channels) for i in range(num_stacks - 1))
        self.merge = nn.ModuleList(Conv(output_channels, channels) for i in range(num_stacks - 1))
    def forward(self, x):
        x = self.prepare(x)
        predicts = []
        for i in range(self.num_stacks):
            x = self.hourglass[i](x)
            predicts.append(self.output[i](x))
            if i != self.num_stacks - 1:
                x = self.merge[i](predicts[-1]) + self.next[i](x)
        return torch.stack(predicts, 0)

def get_model(): # Function to create an object of class model
    return StackedHourglassNet(num_stacks=NUM_STACKS, channels=FIRST_DEPTH_CHANNELS,
                                     output_channels=OUTPUT_CHANNELS,
                                     add_channels=NEXT_DEPTH_ADD_CHANNELS)

def get_train_data():
    cnt = 0
    train_data = []
    train_pos = []
    for _, _, _, left_mag, right_mag, mixed_mag, _, _ in data_generator(train_d,train=True):
        train_data.append((left_mag, right_mag, mixed_mag))
        for i in range(mixed_mag.shape[-1]):
            train_pos.append((cnt, i))
        cnt += 1
    return train_data, train_pos

In [ ]:
#TOTAL NUMBER OF TRAINABLE PARAMETERS
net = get_model()
total_params_SH1 = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(total_params_SH1)

8868038


In [ ]:
ITERATIONS = 1500
BATCH_SIZE = 4  
NUM_STACKS = 1
FIRST_DEPTH_CHANNELS = 64
OUTPUT_CHANNELS = 2  
NEXT_DEPTH_ADD_CHANNELS = 64  
TRAIN_SAVE_POINT = 50  
TEST_STEP = 20 
TOTAL_TEST = 230
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 



def train():
    net = get_model()
    net.to(device)
    training_data, training_indices_array = get_train_data()
    loss_s = torch.empty(1).to(device)
    optimiser = torch.optim.Adam(net.parameters(), lr=0.001)
    for i in tqdm.tqdm(range(ITERATIONS)):
        # declaring input and output tensors
        input = torch.empty(BATCH_SIZE, 1, 512, 64)
        output = torch.empty(BATCH_SIZE, 2, 512, 64)
        
        for j in range(BATCH_SIZE):
            indexing, starting_index = training_indices_array[np.random.randint(len(training_indices_array))]  
            left_mag, right_mag, mixed_mag = training_data[indexing]
            input[j, 0, :, :] = torch.from_numpy(mixed_mag[:512, starting_index:starting_index + 64])
            output[j, 0, :, :] = torch.from_numpy(left_mag[:512, starting_index:starting_index + 64])
            output[j, 1, :, :] = torch.from_numpy(right_mag[:512, starting_index:starting_index + 64])
            input = input.to(device)
            output = output.to(device)
        optimiser.zero_grad()
        predicted = net(input).to(device)
 
        loss = sum(torch.mean(torch.abs(predicted[x].mul(input) - output)) for x in range(1))
        if loss>=0:
          loss.backward()
          loss_s += loss
          optimiser.step()
        if (i + 1) % TRAIN_SAVE_POINT == 0:
            torch.save(net.state_dict(), 'Model1/checkpoint_{}.pt'.format(i + 1))
            print( "loss is {}".format(loss_s / TRAIN_SAVE_POINT,))
            loss_s = 0
    

def test(model='Model1/checkpoint_1500.pt'):
    models = get_model()
    models.load_state_dict(torch.load(model),strict=False)
    models.to(device)
    input = np.empty((BATCH_SIZE, 1, 512, 64), dtype=np.float32)
    length = 0.
    cnt = 0
    gnsdr = 0.
    gsir = 0.
    gsar = 0.
    for left, right, mix, left_magnitude, right_magnitude, mix_magnitude, max_value, mix_phase in data_generator(train_d,train=False):
        source_length = mix_magnitude.shape[-1]
        start_ind = 0
        predicts_left = np.zeros((512, source_length), dtype=np.float32)
        predicts_right = np.zeros((512, source_length), dtype=np.float32)
        
        start = time.time()
        while start_ind + 64 < source_length:
            
            if start_ind and start_ind + (BATCH_SIZE - 1) * 32 + 64 < source_length:
                for i in range(BATCH_SIZE):
                    input[i, 0, :, :] = mix_magnitude[0:512, start_ind + i * 32:start_ind + i * 32 + 64]
                output = models(torch.from_numpy(input).to(device))[-1].data.cpu().numpy() 
                for i in range(BATCH_SIZE):
                    predicts_left[:, start_ind + i * 32 + 16: start_ind + i * 32 + 48] = output[i, 0, :, 16:48]
                    predicts_right[:, start_ind + i * 32 + 16: start_ind + i * 32 + 48] = output[i, 1, :, 16:48]
                start_ind += BATCH_SIZE * 32
            else:
                input[0, 0, :, :] = mix_magnitude[0:512, start_ind:start_ind + 64]
                output = models(torch.from_numpy(input).to(device))[-1].data.cpu().numpy() 
                if start_ind == 0:
                    predicts_left[:, 0:64] = output[0, 0, :, :]
                    predicts_right[:, 0:64] = output[0, 1, :, :]
                else:
                    predicts_left[:, start_ind + 16: start_ind + 48] = output[0, 0, :, 16:48]
                    predicts_right[:, start_ind + 16:start_ind + 48] = output[0, 1, :, 16:48]
                start_ind += 32
        input[0, 0, :, :] = mix_magnitude[0:512, source_length - 64:source_length]
        output = models(torch.from_numpy(input).to(device))[-1].data.cpu().numpy()  
        length = source_length - start_ind - 16
        predicts_left[:, start_ind + 16:source_length] = output[0, 0, :, 64 - length:64]
        predicts_right[:, start_ind + 16:source_length] = output[0, 1, :, 64 - length:64]
        
        predicts_left =np.nan_to_num(predicts_left)
        predicts_right =np.nan_to_num(predicts_right)
        predicts_left[np.where(predicts_left < 0)] = 0
        predicts_right[np.where(predicts_right < 0)] = 0
        predicts_left = predicts_left * mix_magnitude[0:512, :] * max_value
        predicts_right = predicts_right * mix_magnitude[0:512, :] * max_value
        predicts_left_wav = to_wav(predicts_left, mix_phase[0:512, :])
        predicts_right_wav = to_wav(predicts_right, mix_phase[0:512, :])
        predicts_left_wav = librosa.resample(predicts_left_wav, 8000, 16000)
        predicts_right_wav = librosa.resample(predicts_right_wav, 8000, 16000)
        sdr, sir, sar, lens = bss_eval(mix, left, right, predicts_left_wav,
                                              predicts_right_wav)
        length = length + lens
        gnsdr = gnsdr + sdr * lens
        gsir = gsir + sir * lens
        gsar = gsar + sar * lens
        cnt += 1
        if cnt % TEST_STEP == 0:
            print('GNSDR: ', gnsdr / length)
            print('GSIR: ', gsir / length)
            print('GSAR: ', gsar / length)
            
            
        if cnt == TOTAL_TEST:
            break


os.mkdir('Model1')

# New Section

In [ ]:
train()

  3%|▎         | 52/1500 [00:06<03:16,  7.38it/s]

loss of 50 is tensor([-9.1422e+33], device='cuda:0', grad_fn=<DivBackward0>)


  7%|▋         | 102/1500 [00:12<03:00,  7.73it/s]

loss of 100 is 0.007760205771774054


 10%|█         | 152/1500 [00:19<02:54,  7.72it/s]

...

loss of 1400 is 0.004023166839033365


 97%|█████████▋| 1452/1500 [03:10<00:06,  7.28it/s]

loss of 1450 is 0.004071845207363367


100%|██████████| 1500/1500 [03:17<00:00,  7.61it/s]

loss of 1500 is 0.0036650157999247313


In [ ]:
test()

  0%|          | 0/230 [00:00<?, ?it/s]

  9%|▊         | 20/230 [00:40<06:54,  1.97s/it]

GNSDR:  [ 9.29741592 10.42193195]
GSIR:  [12.58453554 16.66428616]
GSAR:  [12.57899469 11.90913728]


 17%|█▋        | 40/230 [01:21<06:42,  2.12s/it]

GNSDR:  [ 9.70749218 11.37613688]
GSIR:  [13.09621566 18.36202007]
GSAR:  [12.84738225 12.59647486]


...

 87%|████████▋ | 200/230 [06:59<01:04,  2.15s/it]

GNSDR:  [9.14837977 9.86141017]
GSIR:  [12.64298148 16.05023346]
GSAR:  [12.1842329  11.39280715]


 96%|█████████▌| 220/230 [07:39<00:18,  1.80s/it]

GNSDR:  [9.08708454 9.79673213]
GSIR:  [12.57614819 15.98145621]
GSAR:  [12.13160477 11.32328618]


100%|██████████| 230/230 [07:58<00:00,  2.08s/it]


#STACKED HOURGLASS - 2 

In [ ]:
NUM_STACKS = 2 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def train():
    net = get_model()
    net.to(device)
    training_data, training_indices_array = get_train_data()
    loss_s = torch.empty(1).to(device)
    optimiser = torch.optim.Adam(net.parameters(), lr=0.001)
    for i in tqdm.tqdm(range(ITERATIONS)):
        # declaring input and output tensors
        input = torch.empty(BATCH_SIZE, 1, 512, 64)
        output = torch.empty(BATCH_SIZE, 2, 512, 64)
        
        for j in range(BATCH_SIZE):
            indexing, starting_index = training_indices_array[np.random.randint(len(training_indices_array))]  
            left_mag, right_mag, mixed_mag = training_data[indexing]
            input[j, 0, :, :] = torch.from_numpy(mixed_mag[:512, starting_index:starting_index + 64])
            output[j, 0, :, :] = torch.from_numpy(left_mag[:512, starting_index:starting_index + 64])
            output[j, 1, :, :] = torch.from_numpy(right_mag[:512, starting_index:starting_index + 64])
            input = input.to(device)
            output = output.to(device)
        optimiser.zero_grad()
        predicted = net(input).to(device)
 
        loss = sum(torch.mean(torch.abs(predicted[x].mul(input) - output)) for x in range(2))
        if loss>=0:
          loss.backward()
          loss_s += loss
          optimiser.step()
        if (i + 1) % TRAIN_SAVE_POINT == 0:
            torch.save(net.state_dict(), 'Model2/checkpoint_{}.pt'.format(i + 1))
            print( "loss is {}".format(loss_s / TRAIN_SAVE_POINT,))
            loss_s = 0

def test(model='Model2/checkpoint_1500.pt'):
    models = get_model()
    models.load_state_dict(torch.load(model),strict=False)
    models.to(device)
    input = np.empty((BATCH_SIZE, 1, 512, 64), dtype=np.float32)
    length = 0.
    cnt = 0
    gnsdr = 0.
    gsir = 0.
    gsar = 0.
    for left, right, mix, left_magnitude, right_magnitude, mix_magnitude, max_value, mix_phase in data_generator(train_d,train=False):
        source_length = mix_magnitude.shape[-1]
        start_ind = 0
        predicts_left = np.zeros((512, source_length), dtype=np.float32)
        predicts_right = np.zeros((512, source_length), dtype=np.float32)
        
        start = time.time()
        while start_ind + 64 < source_length:
            
            if start_ind and start_ind + (BATCH_SIZE - 1) * 32 + 64 < source_length:
                for i in range(BATCH_SIZE):
                    input[i, 0, :, :] = mix_magnitude[0:512, start_ind + i * 32:start_ind + i * 32 + 64]
                output = models(torch.from_numpy(input).to(device))[-1].data.cpu().numpy() 
                for i in range(BATCH_SIZE):
                    predicts_left[:, start_ind + i * 32 + 16: start_ind + i * 32 + 48] = output[i, 0, :, 16:48]
                    predicts_right[:, start_ind + i * 32 + 16: start_ind + i * 32 + 48] = output[i, 1, :, 16:48]
                start_ind += BATCH_SIZE * 32
            else:
                input[0, 0, :, :] = mix_magnitude[0:512, start_ind:start_ind + 64]
                output = models(torch.from_numpy(input).to(device))[-1].data.cpu().numpy() 
                if start_ind == 0:
                    predicts_left[:, 0:64] = output[0, 0, :, :]
                    predicts_right[:, 0:64] = output[0, 1, :, :]
                else:
                    predicts_left[:, start_ind + 16: start_ind + 48] = output[0, 0, :, 16:48]
                    predicts_right[:, start_ind + 16:start_ind + 48] = output[0, 1, :, 16:48]
                start_ind += 32
        input[0, 0, :, :] = mix_magnitude[0:512, source_length - 64:source_length]
        output = models(torch.from_numpy(input).to(device))[-1].data.cpu().numpy()  
        length = source_length - start_ind - 16
        predicts_left[:, start_ind + 16:source_length] = output[0, 0, :, 64 - length:64]
        predicts_right[:, start_ind + 16:source_length] = output[0, 1, :, 64 - length:64]
        
        predicts_left =np.nan_to_num(predicts_left)
        predicts_right =np.nan_to_num(predicts_right)
        predicts_left[np.where(predicts_left < 0)] = 0
        predicts_right[np.where(predicts_right < 0)] = 0
        predicts_left = predicts_left * mix_magnitude[0:512, :] * max_value
        predicts_right = predicts_right * mix_magnitude[0:512, :] * max_value
        predicts_left_wav = to_wav(predicts_left, mix_phase[0:512, :])
        predicts_right_wav = to_wav(predicts_right, mix_phase[0:512, :])
        predicts_left_wav = librosa.resample(predicts_left_wav, 8000, 16000)
        predicts_right_wav = librosa.resample(predicts_right_wav, 8000, 16000)
        sdr, sir, sar, lens = bss_eval(mix, left, right, predicts_left_wav,
                                              predicts_right_wav)
        length = length + lens
        gnsdr = gnsdr + sdr * lens
        gsir = gsir + sir * lens
        gsar = gsar + sar * lens
        cnt += 1
        if cnt % TEST_STEP == 0:
            print('GNSDR: ', gnsdr / length)
            print('GSIR: ', gsir / length)
            print('GSAR: ', gsar / length)
            
            
        if cnt == TOTAL_TEST:
            break

os.mkdir('Model2')

In [ ]:
#TOTAL NUMBER OF TRAINABLE PARAMETERS
net = get_model()
total_params_SH2 = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(total_params_SH2)

17475276


In [ ]:
train()

  3%|▎         | 50/1500 [00:10<06:10,  3.92it/s]

loss of 50 is tensor([0.0195], device='cuda:0', grad_fn=<DivBackward0>)


  7%|▋         | 100/1500 [00:21<06:08,  3.80it/s]

loss of 100 is 0.015661755576729774


 10%|█         | 150/1500 [00:31<05:40,  3.97it/s]

...

loss of 1400 is 0.007266509812325239


 97%|█████████▋| 1450/1500 [05:23<00:13,  3.74it/s]

loss of 1450 is 0.007946201600134373


100%|██████████| 1500/1500 [05:34<00:00,  4.48it/s]

loss of 1500 is 0.0073224701918661594


In [ ]:
test()

  9%|▊         | 20/230 [00:47<07:21,  2.10s/it]

GNSDR:  [ 9.80038162 10.70581223]
GSIR:  [13.92219722 17.90606632]
GSAR:  [12.35646391 11.90790399]


 17%|█▋        | 40/230 [01:28<06:58,  2.20s/it]

GNSDR:  [10.2807994  11.76483737]
GSIR:  [14.51503801 19.34933434]
GSAR:  [12.73278651 12.84248022]


 26%|██▌       | 60/230 [02:11<05:18,  1.88s/it]

...

 87%|████████▋ | 200/230 [07:22<01:06,  2.23s/it]

GNSDR:  [ 9.22554019 10.02901378]
GSIR:  [13.03318357 16.88618576]
GSAR:  [11.98986417 11.39522551]


 96%|█████████▌| 220/230 [08:03<00:18,  1.83s/it]

GNSDR:  [9.14332844 9.93643997]
GSIR:  [12.92863707 16.77771608]
GSAR:  [11.92048093 11.30910882]


100%|██████████| 230/230 [08:23<00:00,  2.19s/it]


#STACKED HOURGLASS - 4

In [ ]:
MAX_ITERATIONS = 500
NUM_STACKS = 4  
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def train():
    net = get_model()
    net.to(device)
    training_data, training_indices_array = get_train_data()
    loss_s = torch.empty(1).to(device)
    optimiser = torch.optim.Adam(net.parameters(), lr=0.001)
    for i in tqdm.tqdm(range(ITERATIONS)):
        # declaring input and output tensors
        input = torch.empty(BATCH_SIZE, 1, 512, 64)
        output = torch.empty(BATCH_SIZE, 2, 512, 64)
        
        for j in range(BATCH_SIZE):
            indexing, starting_index = training_indices_array[np.random.randint(len(training_indices_array))]  
            left_mag, right_mag, mixed_mag = training_data[indexing]
            input[j, 0, :, :] = torch.from_numpy(mixed_mag[:512, starting_index:starting_index + 64])
            output[j, 0, :, :] = torch.from_numpy(left_mag[:512, starting_index:starting_index + 64])
            output[j, 1, :, :] = torch.from_numpy(right_mag[:512, starting_index:starting_index + 64])
            input = input.to(device)
            output = output.to(device)
        optimiser.zero_grad()
        predicted = net(input).to(device)
 
        loss = sum(torch.mean(torch.abs(predicted[x].mul(input) - output)) for x in range(4))
        if loss>=0:
          loss.backward()
          loss_s += loss
          optimiser.step()
        if (i + 1) % TRAIN_SAVE_POINT == 0:
            torch.save(net.state_dict(), 'Model4/checkpoint_{}.pt'.format(i + 1))
            print( "loss is {}".format(loss_s / TRAIN_SAVE_POINT,))
            loss_s = 0
def test(model='Model4/checkpoint_500.pt'):
    models = get_model()
    models.load_state_dict(torch.load(model),strict=False)
    models.to(device)
    input = np.empty((BATCH_SIZE, 1, 512, 64), dtype=np.float32)
    length = 0.
    cnt = 0
    gnsdr = 0.
    gsir = 0.
    gsar = 0.
    for left, right, mix, left_magnitude, right_magnitude, mix_magnitude, max_value, mix_phase in data_generator(train_d,train=False):
        source_length = mix_magnitude.shape[-1]
        start_ind = 0
        predicts_left = np.zeros((512, source_length), dtype=np.float32)
        predicts_right = np.zeros((512, source_length), dtype=np.float32)
        
        start = time.time()
        while start_ind + 64 < source_length:
            
            if start_ind and start_ind + (BATCH_SIZE - 1) * 32 + 64 < source_length:
                for i in range(BATCH_SIZE):
                    input[i, 0, :, :] = mix_magnitude[0:512, start_ind + i * 32:start_ind + i * 32 + 64]
                output = models(torch.from_numpy(input).to(device))[-1].data.cpu().numpy() 
                for i in range(BATCH_SIZE):
                    predicts_left[:, start_ind + i * 32 + 16: start_ind + i * 32 + 48] = output[i, 0, :, 16:48]
                    predicts_right[:, start_ind + i * 32 + 16: start_ind + i * 32 + 48] = output[i, 1, :, 16:48]
                start_ind += BATCH_SIZE * 32
            else:
                input[0, 0, :, :] = mix_magnitude[0:512, start_ind:start_ind + 64]
                output = models(torch.from_numpy(input).to(device))[-1].data.cpu().numpy() 
                if start_ind == 0:
                    predicts_left[:, 0:64] = output[0, 0, :, :]
                    predicts_right[:, 0:64] = output[0, 1, :, :]
                else:
                    predicts_left[:, start_ind + 16: start_ind + 48] = output[0, 0, :, 16:48]
                    predicts_right[:, start_ind + 16:start_ind + 48] = output[0, 1, :, 16:48]
                start_ind += 32
        input[0, 0, :, :] = mix_magnitude[0:512, source_length - 64:source_length]
        output = models(torch.from_numpy(input).to(device))[-1].data.cpu().numpy()  
        length = source_length - start_ind - 16
        predicts_left[:, start_ind + 16:source_length] = output[0, 0, :, 64 - length:64]
        predicts_right[:, start_ind + 16:source_length] = output[0, 1, :, 64 - length:64]
        
        predicts_left =np.nan_to_num(predicts_left)
        predicts_right =np.nan_to_num(predicts_right)
        predicts_left[np.where(predicts_left < 0)] = 0
        predicts_right[np.where(predicts_right < 0)] = 0
        predicts_left = predicts_left * mix_magnitude[0:512, :] * max_value
        predicts_right = predicts_right * mix_magnitude[0:512, :] * max_value
        predicts_left_wav = to_wav(predicts_left, mix_phase[0:512, :])
        predicts_right_wav = to_wav(predicts_right, mix_phase[0:512, :])
        predicts_left_wav = librosa.resample(predicts_left_wav, 8000, 16000)
        predicts_right_wav = librosa.resample(predicts_right_wav, 8000, 16000)
        sdr, sir, sar, lens = bss_eval(mix, left, right, predicts_left_wav,
                                              predicts_right_wav)
        length = length + lens
        gnsdr = gnsdr + sdr * lens
        gsir = gsir + sir * lens
        gsar = gsar + sar * lens
        cnt += 1
        if cnt % TEST_STEP == 0:
            print('GNSDR: ', gnsdr / length)
            print('GSIR: ', gsir / length)
            print('GSAR: ', gsar / length)
            
            
        if cnt == TOTAL_TEST:
            break

In [ ]:
#TOTAL NUMBER OF TRAINABLE PARAMETERS
net = get_model()
total_params_SH4 = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(total_params_SH4)

34689752


In [ ]:
train()

 10%|█         | 50/500 [00:19<03:44,  2.01it/s]

loss of 50 is tensor([0.0396], device='cuda:0', grad_fn=<DivBackward0>)


 20%|██        | 100/500 [00:38<03:19,  2.01it/s]

loss of 100 is 0.030864646658301353


 30%|███       | 150/500 [00:58<02:53,  2.02it/s]

...

loss of 400 is 0.02080450765788555


 90%|█████████ | 450/500 [02:59<00:25,  1.94it/s]

loss of 450 is 0.020517243072390556


100%|██████████| 500/500 [03:19<00:00,  2.50it/s]

loss of 500 is 0.021114736795425415


In [ ]:
test()

  9%|▊         | 20/230 [00:46<07:53,  2.26s/it]

GNSDR:  [8.77572573 9.35972856]
GSIR:  [13.09098554 14.54126495]
GSAR:  [11.25119801 11.35620464]


 17%|█▋        | 40/230 [01:30<07:33,  2.39s/it]

GNSDR:  [ 9.19988281 10.31610165]
GSIR:  [13.44184313 16.37196536]
GSAR:  [11.69999143 11.97317494]


 26%|██▌       | 60/230 [02:16<05:45,  2.03s/it]

...

 87%|████████▋ | 200/230 [07:50<01:11,  2.40s/it]

GNSDR:  [8.13190496 8.81071897]
GSIR:  [11.86577725 14.99643082]
GSAR:  [11.10396898 10.47688817]


 96%|█████████▌| 220/230 [08:34<00:19,  1.97s/it]

GNSDR:  [8.0546265  8.74925667]
GSIR:  [11.76520215 14.95189058]
GSAR:  [11.04173583 10.40547879]


100%|██████████| 230/230 [08:56<00:00,  2.33s/it]


#CNN



In [ ]:
class CNN(nn.Module):
  def __init__(self):
        super(CNN, self).__init__()
        self.model = nn.Sequential(
                Conv(1, 64, kernel_size=7, stride=1),
                Conv(64, 128), 
                Conv(128, 128),
                Conv(128, 256),
                Conv(256, 2)
                )

  def forward(self, x):
      return self.model(x)

def get_model(): # Function to create an object of class model
    return CNN()

In [ ]:
MAX_ITERATIONS = 500
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def train():
    net = get_model()
    net.to(device)
    training_data, training_indices_array = get_train_data()
    loss_s = torch.empty(1).to(device)
    optimiser = torch.optim.Adam(net.parameters(), lr=0.001)
    for i in tqdm.tqdm(range(ITERATIONS)):
        # declaring input and output tensors
        input = torch.empty(BATCH_SIZE, 1, 512, 64)
        output = torch.empty(BATCH_SIZE, 2, 512, 64)
        
        for j in range(BATCH_SIZE):
            indexing, starting_index = training_indices_array[np.random.randint(len(training_indices_array))]  
            left_mag, right_mag, mixed_mag = training_data[indexing]
            input[j, 0, :, :] = torch.from_numpy(mixed_mag[:512, starting_index:starting_index + 64])
            output[j, 0, :, :] = torch.from_numpy(left_mag[:512, starting_index:starting_index + 64])
            output[j, 1, :, :] = torch.from_numpy(right_mag[:512, starting_index:starting_index + 64])
            input = input.to(device)
            output = output.to(device)
        optimiser.zero_grad()
        predicted = net(input).to(device)
 
        loss = sum(torch.mean(torch.abs(predicted[x].mul(input) - output)) for x in range(4))
        if loss>=0:
          loss.backward()
          loss_s += loss
          optimiser.step()
        if (i + 1) % TRAIN_SAVE_POINT == 0:
            torch.save(net.state_dict(), 'ModelCNN/checkpoint_{}.pt'.format(i + 1))
            print( "loss is {}".format(loss_s / TRAIN_SAVE_POINT,))
            loss_s = 0
def test(model='ModelCNN/checkpoint_500.pt'):
    models = get_model()
    models.load_state_dict(torch.load(model),strict=False)
    models.to(device)
    input = np.empty((BATCH_SIZE, 1, 512, 64), dtype=np.float32)
    length = 0.
    cnt = 0
    gnsdr = 0.
    gsir = 0.
    gsar = 0.
    for left, right, mix, left_magnitude, right_magnitude, mix_magnitude, max_value, mix_phase in data_generator(train_d,train=False):
        source_length = mix_magnitude.shape[-1]
        start_ind = 0
        predicts_left = np.zeros((512, source_length), dtype=np.float32)
        predicts_right = np.zeros((512, source_length), dtype=np.float32)
        
        start = time.time()
        while start_ind + 64 < source_length:
            
            if start_ind and start_ind + (BATCH_SIZE - 1) * 32 + 64 < source_length:
                for i in range(BATCH_SIZE):
                    input[i, 0, :, :] = mix_magnitude[0:512, start_ind + i * 32:start_ind + i * 32 + 64]
                output = models(torch.from_numpy(input).to(device))[-1].data.cpu().numpy() 
                for i in range(BATCH_SIZE):
                    predicts_left[:, start_ind + i * 32 + 16: start_ind + i * 32 + 48] = output[i, 0, :, 16:48]
                    predicts_right[:, start_ind + i * 32 + 16: start_ind + i * 32 + 48] = output[i, 1, :, 16:48]
                start_ind += BATCH_SIZE * 32
            else:
                input[0, 0, :, :] = mix_magnitude[0:512, start_ind:start_ind + 64]
                output = models(torch.from_numpy(input).to(device))[-1].data.cpu().numpy() 
                if start_ind == 0:
                    predicts_left[:, 0:64] = output[0, 0, :, :]
                    predicts_right[:, 0:64] = output[0, 1, :, :]
                else:
                    predicts_left[:, start_ind + 16: start_ind + 48] = output[0, 0, :, 16:48]
                    predicts_right[:, start_ind + 16:start_ind + 48] = output[0, 1, :, 16:48]
                start_ind += 32
        input[0, 0, :, :] = mix_magnitude[0:512, source_length - 64:source_length]
        output = models(torch.from_numpy(input).to(device))[-1].data.cpu().numpy()  
        length = source_length - start_ind - 16
        predicts_left[:, start_ind + 16:source_length] = output[0, 0, :, 64 - length:64]
        predicts_right[:, start_ind + 16:source_length] = output[0, 1, :, 64 - length:64]
        
        predicts_left =np.nan_to_num(predicts_left)
        predicts_right =np.nan_to_num(predicts_right)
        predicts_left[np.where(predicts_left < 0)] = 0
        predicts_right[np.where(predicts_right < 0)] = 0
        predicts_left = predicts_left * mix_magnitude[0:512, :] * max_value
        predicts_right = predicts_right * mix_magnitude[0:512, :] * max_value
        predicts_left_wav = to_wav(predicts_left, mix_phase[0:512, :])
        predicts_right_wav = to_wav(predicts_right, mix_phase[0:512, :])
        predicts_left_wav = librosa.resample(predicts_left_wav, 8000, 16000)
        predicts_right_wav = librosa.resample(predicts_right_wav, 8000, 16000)
        sdr, sir, sar, lens = bss_eval(mix, left, right, predicts_left_wav,
                                              predicts_right_wav)
        length = length + lens
        gnsdr = gnsdr + sdr * lens
        gsir = gsir + sir * lens
        gsar = gsar + sar * lens
        cnt += 1
        if cnt % TEST_STEP == 0:
            print('GNSDR: ', gnsdr / length)
            print('GSIR: ', gsir / length)
            print('GSAR: ', gsar / length)
            
            
        if cnt == TOTAL_TEST:
            break

os.mkdir('ModelCNN')

In [ ]:
#TOTAL NUMBER OF TRAINABLE PARAMETERS
net = get_model()
total_params_CNN = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(total_params_CNN)

In [ ]:
train()

 10%|█         | 51/500 [00:22<03:09,  2.37it/s]

loss of 50 is tensor([5.0581e+17], device='cuda:0', grad_fn=<DivBackward0>)


 20%|██        | 101/500 [00:44<02:45,  2.41it/s]

loss of 100 is 0.04187924414873123


 30%|███       | 151/500 [01:05<02:25,  2.40it/s]

...

loss of 400 is 0.03653066232800484


 90%|█████████ | 451/500 [03:15<00:20,  2.42it/s]

loss of 450 is 0.037184394896030426


100%|██████████| 500/500 [03:37<00:00,  2.30it/s]

loss of 500 is 0.03388440981507301


In [ ]:
test()


  9%|▊         | 20/230 [01:04<11:53,  3.40s/it]

GNSDR:  [-5.87881972  0.47665181]
GSIR:  [-0.83845632  3.30814261]
GSAR:  [-0.25534847  5.94358982]



 17%|█▋        | 40/230 [02:04<08:39,  2.73s/it]

GNSDR:  [-5.5543744   0.08443792]
GSIR:  [-0.27965419  3.05156787]
GSAR:  [-0.17814215  5.73585023]



 26%|██▌       | 60/230 [03:08<08:24,  2.97s/it]

...


 78%|███████▊  | 180/230 [09:36<02:21,  2.84s/it]

GNSDR:  [-6.31167148  0.73460041]
GSIR:  [-2.03101033  4.66975966]
GSAR:  [0.84885052 5.11172873]



 87%|████████▋ | 200/230 [10:42<01:45,  3.50s/it]

GNSDR:  [-6.32929306  0.78525739]
GSIR:  [-2.06630626  4.71751401]
GSAR:  [0.82717496 5.10445043]



 92%|█████████▏| 211/230 [11:21<01:01,  3.23s/it]


#STACKED HOURGLASS 4 - 1500 ITERATIONS

In [ ]:
MAX_ITERATIONS = 1500
NUM_STACKS = 4  
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def train():
    net = get_model()
    net.to(device)
    training_data, training_indices_array = get_train_data()
    loss_s = torch.empty(1).to(device)
    optimiser = torch.optim.Adam(net.parameters(), lr=0.001)
    for i in tqdm.tqdm(range(ITERATIONS)):
        # declaring input and output tensors
        input = torch.empty(BATCH_SIZE, 1, 512, 64)
        output = torch.empty(BATCH_SIZE, 2, 512, 64)
        
        for j in range(BATCH_SIZE):
            indexing, starting_index = training_indices_array[np.random.randint(len(training_indices_array))]  
            left_mag, right_mag, mixed_mag = training_data[indexing]
            input[j, 0, :, :] = torch.from_numpy(mixed_mag[:512, starting_index:starting_index + 64])
            output[j, 0, :, :] = torch.from_numpy(left_mag[:512, starting_index:starting_index + 64])
            output[j, 1, :, :] = torch.from_numpy(right_mag[:512, starting_index:starting_index + 64])
            input = input.to(device)
            output = output.to(device)
        optimiser.zero_grad()
        predicted = net(input).to(device)
 
        loss = sum(torch.mean(torch.abs(predicted[x].mul(input) - output)) for x in range(1))
        if loss>=0:
          loss.backward()
          loss_s += loss
          optimiser.step()
        if (i + 1) % TRAIN_SAVE_POINT == 0:
            torch.save(net.state_dict(), 'Model4(1)/checkpoint_{}.pt'.format(i + 1))
            print( "loss is {}".format(loss_s / TRAIN_SAVE_POINT,))
            loss_s = 0
    
def test(model='Model4(1)/checkpoint_1500.pt'):
    models = get_model()
    models.load_state_dict(torch.load(model),strict=False)
    models.to(device)
    input = np.empty((BATCH_SIZE, 1, 512, 64), dtype=np.float32)
    length = 0.
    cnt = 0
    gnsdr = 0.
    gsir = 0.
    gsar = 0.
    for left, right, mix, left_magnitude, right_magnitude, mix_magnitude, max_value, mix_phase in data_generator(train_d,train=False):
        source_length = mix_magnitude.shape[-1]
        start_ind = 0
        predicts_left = np.zeros((512, source_length), dtype=np.float32)
        predicts_right = np.zeros((512, source_length), dtype=np.float32)
        
        start = time.time()
        while start_ind + 64 < source_length:
            
            if start_ind and start_ind + (BATCH_SIZE - 1) * 32 + 64 < source_length:
                for i in range(BATCH_SIZE):
                    input[i, 0, :, :] = mix_magnitude[0:512, start_ind + i * 32:start_ind + i * 32 + 64]
                output = models(torch.from_numpy(input).to(device))[-1].data.cpu().numpy() 
                for i in range(BATCH_SIZE):
                    predicts_left[:, start_ind + i * 32 + 16: start_ind + i * 32 + 48] = output[i, 0, :, 16:48]
                    predicts_right[:, start_ind + i * 32 + 16: start_ind + i * 32 + 48] = output[i, 1, :, 16:48]
                start_ind += BATCH_SIZE * 32
            else:
                input[0, 0, :, :] = mix_magnitude[0:512, start_ind:start_ind + 64]
                output = models(torch.from_numpy(input).to(device))[-1].data.cpu().numpy() 
                if start_ind == 0:
                    predicts_left[:, 0:64] = output[0, 0, :, :]
                    predicts_right[:, 0:64] = output[0, 1, :, :]
                else:
                    predicts_left[:, start_ind + 16: start_ind + 48] = output[0, 0, :, 16:48]
                    predicts_right[:, start_ind + 16:start_ind + 48] = output[0, 1, :, 16:48]
                start_ind += 32
        input[0, 0, :, :] = mix_magnitude[0:512, source_length - 64:source_length]
        output = models(torch.from_numpy(input).to(device))[-1].data.cpu().numpy()  
        length = source_length - start_ind - 16
        predicts_left[:, start_ind + 16:source_length] = output[0, 0, :, 64 - length:64]
        predicts_right[:, start_ind + 16:source_length] = output[0, 1, :, 64 - length:64]
        
        predicts_left =np.nan_to_num(predicts_left)
        predicts_right =np.nan_to_num(predicts_right)
        predicts_left[np.where(predicts_left < 0)] = 0
        predicts_right[np.where(predicts_right < 0)] = 0
        predicts_left = predicts_left * mix_magnitude[0:512, :] * max_value
        predicts_right = predicts_right * mix_magnitude[0:512, :] * max_value
        predicts_left_wav = to_wav(predicts_left, mix_phase[0:512, :])
        predicts_right_wav = to_wav(predicts_right, mix_phase[0:512, :])
        predicts_left_wav = librosa.resample(predicts_left_wav, 8000, 16000)
        predicts_right_wav = librosa.resample(predicts_right_wav, 8000, 16000)
        sdr, sir, sar, lens = bss_eval(mix, left, right, predicts_left_wav,
                                              predicts_right_wav)
        length = length + lens
        gnsdr = gnsdr + sdr * lens
        gsir = gsir + sir * lens
        gsar = gsar + sar * lens
        cnt += 1
        if cnt % TEST_STEP == 0:
            print('GNSDR: ', gnsdr / length)
            print('GSIR: ', gsir / length)
            print('GSAR: ', gsar / length)
            #SAVING THE AUDIO FILES - FOR DEMO PURPOSE       
            soundfile.write('/content/drive/MyDrive/test_wavfiles/{}_accompaniments_predict.wav'.format(cnt), predicts_left_wav, 16000, format='wav',subtype='PCM_16')
            soundfile.write('/content/drive/MyDrive/test_wavfiles/{}_voice_predict.wav'.format(cnt), predicts_right_wav, 16000, format='wav',subtype='PCM_16')
            soundfile.write('/content/drive/MyDrive/test_wavfiles/{}mixed.wav'.format(cnt), mix, 16000, format='wav',subtype='PCM_16')
        if cnt == TOTAL_TEST:
            break


In [ ]:
train()

  3%|▎         | 50/1500 [00:19<12:43,  1.90it/s]

loss of 50 is tensor([0.0393], device='cuda:0', grad_fn=<DivBackward0>)


  7%|▋         | 100/1500 [00:38<15:08,  1.54it/s]

loss of 100 is 0.02935784123837948


 10%|█         | 150/1500 [00:57<12:16,  1.83it/s]

...

loss of 1400 is 0.014679123647511005


 97%|█████████▋| 1450/1500 [09:44<00:28,  1.72it/s]

loss of 1450 is 0.014274033717811108


100%|██████████| 1500/1500 [10:05<00:00,  2.48it/s]

loss of 1500 is 0.014869502745568752


In [ ]:
test()

  9%|▊         | 20/230 [00:44<07:33,  2.16s/it]

GNSDR:  [7.77796867 8.73091081]
GSIR:  [10.84189001 16.81503898]
GSAR:  [11.24641794  9.65255729]


 17%|█▋        | 40/230 [01:36<07:47,  2.46s/it]

GNSDR:  [6.9511877  7.65953761]
GSIR:  [10.13924474 15.61238917]
GSAR:  [10.30863059  8.63456765]


 26%|██▌       | 60/230 [02:27<07:54,  2.79s/it]

...

 87%|████████▋ | 200/230 [08:29<01:08,  2.29s/it]

GNSDR:  [7.86381275 8.87897298]
GSIR:  [11.21312622 14.49078417]
GSAR:  [11.62958388 10.86449444]


 96%|█████████▌| 220/230 [09:12<00:20,  2.09s/it]

GNSDR:  [8.03539315 8.95607098]
GSIR:  [11.52434767 14.45645287]
GSAR:  [11.66143523 10.98371314]


100%|██████████| 230/230 [09:34<00:00,  2.50s/it]
